# 1. Importing libraries

In [1]:
import pandas as pd
import numpy as np
import os

# 2. Importing data

In [2]:
#create path
path = r'C:\Users\Mathilde\Documents\DATA ANALYSIS CAREERFOUNDRY\May 2024 - Instacart Basket Analysis'
#orders data
df_ords = pd.read_csv(os.path.join(path, '02 Data', 'Prepared data', 'orders_wrangled.csv'), index_col = False)
#products data
df_prods = pd.read_csv(os.path.join(path, '02 Data', 'Original data', 'products.csv'), index_col = False)
#departments data
df_dep = pd.read_csv(os.path.join(path, '02 Data', 'Prepared data', 'departments_wrangled.csv'), index_col = False)

In [3]:
import warnings
warnings.filterwarnings('ignore')

# 3. Data consistency check on Products dataframe

In [4]:
#Checking for missing values
df_prods.isnull().sum()

product_id        0
product_name     16
aisle_id          0
department_id     0
prices            0
dtype: int64

In [5]:
#View the missing values
df_nan=df_prods[df_prods['product_name'].isnull()==True]
df_nan

,product_id,product_name,aisle_id,department_id,prices
33,34,NaN,121,14,12.2
68,69,NaN,26,7,11.8
115,116,NaN,93,3,10.8
261,262,NaN,110,13,12.1
525,525,NaN,109,11,1.2
1511,1511,NaN,84,16,14.3
1780,1780,NaN,126,11,12.3
2240,2240,NaN,52,1,14.2
2586,2586,NaN,104,13,12.4
3159,3159,NaN,126,11,13.1


In [6]:
#Creata a data frame without the missing values
df_prods_clean=df_prods[df_prods['product_name'].isnull()==False]
#checking if the missing values have been removed by viewing size of new df
df_prods_clean.shape

(49677, 5)

In [7]:
#checking size of original df
df_prods.shape

(49693, 5)

In [8]:
#viewing duplicates
df_dups = df_prods_clean[df_prods_clean.duplicated()]
df_dups

,product_id,product_name,aisle_id,department_id,prices
462,462,Fiber 4g Gummy Dietary Supplement,70,11,4.8
18459,18458,Ranger IPA,27,5,9.2
26810,26808,Black House Coffee Roasty Stout Beer,27,5,13.4
35309,35306,Gluten Free Organic Peanut Butter & Chocolate ...,121,14,6.8
35495,35491,Adore Forever Body Wash,127,11,9.9


In [9]:
df_prods_clean.shape

(49677, 5)

In [10]:
df_prods_clean_no_dups = df_prods_clean.drop_duplicates()

In [11]:
df_prods_clean_no_dups.shape

(49672, 5)

### Checking for inconsistencies

In [12]:
df_prods.describe()

,product_id,aisle_id,department_id,prices
count,49693.000000,49693.000000,49693.000000,49693.000000
mean,24844.345139,67.770249,11.728433,9.994136
std,14343.717401,38.316774,5.850282,453.519686
min,1.000000,1.000000,1.000000,1.000000
25%,12423.000000,35.000000,7.000000,4.100000
50%,24845.000000,69.000000,13.000000,7.100000
75%,37265.000000,100.000000,17.000000,11.200000
max,49688.000000,134.000000,21.000000,99999.000000


### Observation:

Maximum value for price is 99999, which is very much higher than the mean (and therefore, the std is also very high). This is not a price expected for a supermarket product. Let's investigate further. Let's consider no product in a supermarket would cost over 100 dollars.

In [13]:
#identify products with abnormally high price
df_prods_clean_no_dups.loc[df_prods['prices'] > 100]

,product_id,product_name,aisle_id,department_id,prices
21554,21553,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0
33666,33664,2 % Reduced Fat Milk,84,16,99999.0


We found a pack of cottage cheese for 14900 and a pack of milk for 99999 dollars, which are very obviouslu the result of typos or data corruption. Let's assume that a comma has been ommited and correct it:

In [14]:
#replacing outliers 
df_prods_clean_no_dups = df_prods_clean_no_dups.replace ({"prices":{99999.0: 9.99, 14900.0:1.49 }})

In [15]:
df_prods_clean_no_dups.describe()

,product_id,aisle_id,department_id,prices
count,49672.000000,49672.000000,49672.000000,49672.000000
mean,24850.349775,67.762442,11.728942,7.680359
std,14340.705287,38.315784,5.850779,4.199401
min,1.000000,1.000000,1.000000,1.000000
25%,12432.750000,35.000000,7.000000,4.100000
50%,24850.500000,69.000000,13.000000,7.100000
75%,37268.250000,100.000000,17.000000,11.100000
max,49688.000000,134.000000,21.000000,25.000000


# 4. Changing data type

In [16]:
#changing data type for product_id, aisle_id and department_id from int to string
df_prods_clean_no_dups['product_id'] = df_prods_clean_no_dups['product_id'].astype('str')
df_prods_clean_no_dups['aisle_id'] = df_prods_clean_no_dups['aisle_id'].astype('str')
df_prods_clean_no_dups['department_id']= df_prods_clean_no_dups['department_id'].astype('str')

# 5. Export cleaned product data

In [17]:
#export cleaned prods data frame
df_prods_clean_no_dups.to_csv(os.path.join(path, '02 Data','Prepared Data', 'products_checked.csv'))

# 6. Task - Data consistency check on Orders data frame

In [18]:
#Dropping the useless column that showed up
df_ords=df_ords.drop(columns=['Unnamed: 0'])

## 6.2 Checking for inconsistencies in Orders DF

In [19]:
df_ords.describe()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


### Observation:

Day of the week ranges from 0 to 6 - this is normal, 7 days in a week.
Hour of the day ranges from 0 to 23 - also expected
Days since prior order ranges from 0 to 30 - also seems coherent.

## 6.3 Checking for mixed-type data in Orders DF

In [20]:
for col in df_ords.columns.tolist():
  weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_ords[weird]) > 0:
    print (col)

### Observation:

No column returned means that there is no mix-typed data.

## 6.5 Checking for missing values in Orders DF

In [21]:
df_ords.isnull().sum()

order_id                       0
user_id                        0
order_number                   0
order_dow                      0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

### Observation:

It is expected to have missing values in the column "days since prior order": every customer has a first order, for which there is no prior order. The number of missing values should actually match the number of customers (as each customer has one first order)

In [22]:
df_ords.tail(5)

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
3421078,2266710,206209,10,5,18,29.0
3421079,1854736,206209,11,4,10,30.0
3421080,626363,206209,12,1,12,18.0
3421081,2977660,206209,13,1,12,7.0
3421082,272231,206209,14,6,14,30.0


The last user_id is 206209, meaning there are 206209 customers: which is exactly the number of missing values. This is coherent. No action taken.

I found another way of checking this, as the column user_id might not be sorted in descending order:

In [23]:
df_ords['user_id'].nunique()

206209

## 6.7 Checking for duplicates in Orders DF

In [24]:
#view duplicates
df_ords[df_ords.duplicated()]

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order


### Observation:

The resulting view is empty: no duplicates. No action taken

###### No change made to the orders data frame, so I am not exporting it.

In [25]:
df_ords.shape

(3421083, 6)